
# Sector-Constrained Bayesian Index Tracker for the S&P 500 — A Hands-On Tutorial

**Last updated:** 2025-08-09

Welcome! This notebook walks you through building a **sector-constrained, Bayesian-regularized index tracker** that approximates the **S&P 500** using only the sectors you choose.

You will:
- Scrape **S&P 500 constituents + GICS sectors** from Wikipedia.
- Choose which **sectors** to invest in (e.g., Information Technology, Health Care, Financials).
- Fetch **historical prices** for your universe and the S&P 500 index (^GSPC).
- Estimate a **Bayesian ridge** (MAP) regularization strength from data.
- Optionally **limit the number of names** (cardinality) using a greedy forward-selection heuristic.
- Solve a **convex optimization** (quadratic program) to minimize tracking error with:
  - **Shrinkage** (L2, Bayesian MAP),
  - A **sector mix penalty** that nudges your portfolio toward a target sector distribution,
  - **Long-only**, **sum of weights = 1**, and **per-name caps** constraints.
- Evaluate **6-month out-of-sample (OOS)** tracking and build a simple **6‑month forecast**.

> **Note:** This is for research and education. Not investment advice.



## 0. Requirements

Run the cell below to install (or confirm) the required packages.

- `pandas`, `numpy`: data wrangling
- `yfinance`: prices and market caps (Yahoo Finance)
- `cvxpy`, `osqp`, `scs`: convex optimization (QP solver)
- `scikit-learn`: `BayesianRidge` for evidence-based regularization
- `requests`, `lxml`: Wikipedia scraping (via `pandas.read_html`)
- `matplotlib`: charts (we will not set explicit colors)
- `statsmodels`: simple AR(1) forecast
- `tqdm`: progress bars


In [ ]:

# If running on a fresh environment, uncomment this cell and run it once.
# !pip install -q pandas numpy yfinance cvxpy osqp scikit-learn lxml requests tqdm matplotlib statsmodels



## 1. Imports and global configuration

We also set a **lookback** window and an **out-of-sample** (OOS) window.


In [ ]:

import os
import json
import math
import warnings
import datetime as dt
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
import requests
import yfinance as yf
import cvxpy as cp
from sklearn.linear_model import BayesianRidge
from tqdm import tqdm
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# ---- Reproducibility-ish (does not fix all randomness across solvers) ----
np.random.seed(42)

# ---- Config ----
LOOKBACK_YEARS = 3       # how much history to fit on
OOS_MONTHS     = 6       # evaluate the next ~6 months of data
FREQUENCY      = "1d"    # daily data
INDEX_SYMBOL   = "^GSPC" # S&P 500 (Yahoo Finance symbol)
UNIVERSE_CAP   = 300     # keep top-N by market cap in your selected sectors
MAX_NAMES      = 40      # limit the number of names via greedy selection
PER_NAME_CAP   = 0.08    # max weight per stock (e.g., 8%)
LONG_ONLY      = True    # long-only or allow shorting
SECTOR_PENALTY = 5.0     # gamma: strength of sector-mix penalty (0 = off)
L2_FLOOR       = 1e-6    # minimal ridge regularization
OUTPUT_DIR     = "./notebook_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Choose your sectors (GICS names)
ALLOWED_SECTORS = [
    "Information Technology",
    "Health Care",
    "Financials",
    # You can add more:
    # "Communication Services","Consumer Discretionary","Consumer Staples",
    # "Energy","Industrials","Materials","Real Estate","Utilities"
]

# Date range for downloads
today = dt.date.today()
END_DATE = today
START_DATE = END_DATE - dt.timedelta(days=int(365 * LOOKBACK_YEARS + 365))  # +1y buffer
print(f"Training window start: {START_DATE}, end: {END_DATE}")



## 2. Get the S&P 500 constituents and sectors (Wikipedia)

We use `pandas.read_html` to read the constituents table and then normalize tickers for Yahoo Finance (e.g., `BRK.B → BRK-B`).


In [ ]:

WIKI_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

def get_sp500_constituents() -> pd.DataFrame:
    """Return a dataframe with columns: Ticker, Name, Sector, SubIndustry."""
    html = requests.get(WIKI_URL, timeout=30).text
    tables = pd.read_html(html)
    df = tables[0].rename(columns={
        "Symbol": "Ticker",
        "Security": "Name",
        "GICS Sector": "Sector",
        "GICS Sub-Industry": "SubIndustry"
    })
    # Normalize Yahoo tickers (e.g., BRK.B -> BRK-B)
    df["Ticker"] = df["Ticker"].str.replace(".", "-", regex=False).str.upper().str.strip()
    return df[["Ticker", "Name", "Sector", "SubIndustry"]]

sp500_all = get_sp500_constituents()
sp500_all.head()



## 3. Filter by your chosen sectors

We will invest only in the sectors you selected above in `ALLOWED_SECTORS`.


In [ ]:

sp500 = sp500_all[sp500_all["Sector"].isin(ALLOWED_SECTORS)].copy()
if sp500.empty:
    raise ValueError("No tickers found in the selected sectors. Please adjust ALLOWED_SECTORS.")
print(f"Selected sectors: {sorted(set(sp500['Sector']))}")
print(f"Tickers in selected sectors: {len(sp500)}")
sp500.head()



## 4. Fetch market caps and compute sector targets

We approximate sector weights by summing live **market caps** for each sector and normalizing them to 1.  
This is a proxy for float-adjusted market-cap weights (it is not exact).


In [ ]:

def fetch_market_caps(tickers: List[str]) -> pd.Series:
    """Fetch (approximate) market caps for tickers using yfinance.
    Returns a Series indexed by ticker with values as floats.
    """
    caps = {}
    for t in tqdm(tickers, desc="Fetching market caps"):
        try:
            tk = yf.Ticker(t)
            cap = None
            # Try fast_info first
            try:
                cap = tk.fast_info.get("market_cap", None)
            except Exception:
                pass
            if cap is None:
                info = tk.info
                cap = info.get("marketCap", None)
            if cap is not None and np.isfinite(cap):
                caps[t] = float(cap)
        except Exception:
            continue
    return pd.Series(caps, name="MarketCap", dtype="float64")

# Compute sector targets using ALL S&P 500 (more realistic), or only selected sectors.
# Here we use the full list to estimate the reference mix:
caps_reference = fetch_market_caps(sp500_all["Ticker"].tolist())
ref_df = sp500_all.merge(caps_reference.rename("MarketCap"),
                         left_on="Ticker", right_index=True, how="left").dropna(subset=["MarketCap"])
sector_caps_ref = ref_df.groupby("Sector")["MarketCap"].sum()
sector_target = (sector_caps_ref / sector_caps_ref.sum()).sort_values(ascending=False)

# Now attach caps to our selected-universe tickers
caps_selected = fetch_market_caps(sp500["Ticker"].tolist())
sp500 = sp500.merge(caps_selected.rename("MarketCap"),
                    left_on="Ticker", right_index=True, how="left").dropna(subset=["MarketCap"])

print("Reference sector target (top 5 shown):")
sector_target.head()



## 5. Cap the investable universe by size (optional)

To keep downloads and optimization fast, we restrict to the top **UNIVERSE_CAP** names by market cap within your chosen sectors.


In [ ]:

sp500 = sp500.sort_values("MarketCap", ascending=False).head(UNIVERSE_CAP).reset_index(drop=True)
print(f"Universe size after capping: {len(sp500)}")
sp500.head()



## 6. Download historical prices

We download **adjusted close** prices for the S&P 500 index (`^GSPC`) and for our universe.  
We then compute **daily log returns** and split into **in-sample** and **out-of-sample** windows.


In [ ]:

def fetch_prices(tickers: List[str], start, end, interval="1d") -> pd.DataFrame:
    """Batch download adjusted close prices via yfinance.
    Returns a DataFrame with dates as index and tickers as columns.
    """
    data = yf.download(tickers, start=start, end=end, interval=interval,
                       auto_adjust=True, progress=False)
    px = data["Close"] if "Close" in data else data
    if isinstance(px, pd.Series):
        px = px.to_frame(tickers[0])
    return px.dropna(how="all")

tickers = sp500["Ticker"].tolist()
px_assets = fetch_prices(tickers, START_DATE, END_DATE, FREQUENCY)
px_index  = fetch_prices([INDEX_SYMBOL], START_DATE, END_DATE, FREQUENCY)

# Align dates and drop columns with missing data
common_idx = px_assets.index.intersection(px_index.index)
px_assets = px_assets.loc[common_idx].dropna(axis=1, how="any")
px_index  = px_index.loc[common_idx]

# Align universe to available data
available = [t for t in tickers if t in px_assets.columns]
sp500 = sp500[sp500["Ticker"].isin(available)].reset_index(drop=True)
tickers = sp500["Ticker"].tolist()

def log_returns(px: pd.DataFrame) -> pd.DataFrame:
    return np.log(px).diff().dropna()

R = log_returns(px_assets)       # T x N asset log-returns
r_idx = log_returns(px_index)    # T x 1 index log-returns
r_idx = r_idx.iloc[:, 0]         # convert to Series

# Split IS/OOS
oos_days = int(30.42 * OOS_MONTHS)  # ~ trading days metric
if len(R) < oos_days + 252:
    raise ValueError("Not enough history for requested OOS. Increase LOOKBACK_YEARS or reduce OOS_MONTHS.")

R_is   = R.iloc[:-oos_days, :]
R_oos  = R.iloc[-oos_days:, :]
y_is   = r_idx.iloc[:-oos_days]
y_oos  = r_idx.iloc[-oos_days:]
R_is.shape, R_oos.shape



## 7. Build the sector aggregation matrix

We need a matrix **A** with shape `(num_sectors, num_stocks)` where `A[s, j] = 1` if stock `j` is in sector `s`, else 0.  
This allows us to penalize deviations of the **portfolio sector weights** from the **target sector mix**.


In [ ]:

# Sectors present in our (post-cleaning) universe
sectors = sorted(sp500["Sector"].unique())
S = len(sectors)
sector_to_row = {s: i for i, s in enumerate(sectors)}

# Build A for the FULL universe (we will rebuild it again after selection)
N_full = len(tickers)
A_full = np.zeros((S, N_full), dtype=float)
for j, t in enumerate(tickers):
    s = sp500.loc[sp500["Ticker"] == t, "Sector"].values[0]
    A_full[sector_to_row[s], j] = 1.0

# Target vector aligned to `sectors`
s_target = np.array([sector_target.get(s, 0.0) for s in sectors], dtype=float)
if s_target.sum() > 0:
    s_target = s_target / s_target.sum()

S, N_full, s_target[:5]



## 8. Estimate Bayesian ridge regularization (λ) via evidence maximization

We fit `BayesianRidge` on **standardized** in-sample returns to obtain an effective L2 penalty.  
This gives a **MAP** solution equivalent to ridge regression, which stabilizes the portfolio weights.


In [ ]:

def estimate_lambda_bayesian(R: pd.DataFrame, y: pd.Series, l2_floor: float = 1e-6) -> float:
    X = R.values
    Y = y.values
    # Standardize for evidence stability (the QP will use unstandardized returns)
    Xs = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-12)
    Ys = (Y - Y.mean()) / (Y.std() + 1e-12)
    br = BayesianRidge(compute_score=True, fit_intercept=True)
    br.fit(Xs, Ys)
    lam = float(br.lambda_ / max(br.alpha_, 1e-12))
    return max(lam, l2_floor)

ridge_lambda = estimate_lambda_bayesian(R_is, y_is, L2_FLOOR)
print(f"Estimated ridge lambda (Bayesian evidence): {ridge_lambda:.4g}")



## 9. Greedy forward selection (optional) to limit the number of names

Exact cardinality constraints make the problem **mixed-integer** and slow.  
Instead, we use a **greedy** heuristic: iteratively add the stock most correlated with the **current residual** vs the index.


In [ ]:

def greedy_select(R: pd.DataFrame, y: pd.Series, lam: float, K: int | None) -> List[str]:
    if K is None or K >= R.shape[1]:
        return list(R.columns)

    chosen = []
    remaining = set(R.columns)

    # Start with the max |corr| to index
    corr = R.corrwith(y).abs().sort_values(ascending=False)
    first = corr.index[0]
    chosen.append(first)
    remaining.remove(first)

    for _ in range(1, K):
        # Fit ridge on current chosen to get residuals
        Xc = R[chosen].values
        yc = y.values
        XtX = Xc.T @ Xc
        w = np.linalg.solve(XtX + lam*np.eye(len(chosen)), Xc.T @ yc)
        resid = yc - Xc @ w

        # Pick remaining name with highest |corr| to residual
        best_t, best_score = None, -1.0
        for t in remaining:
            x = R[t].values
            num = np.dot(resid - resid.mean(), x - x.mean())
            den = np.sqrt(((resid - resid.mean())**2).sum() * ((x - x.mean())**2).sum()) + 1e-18
            c = abs(num) / den
            if c > best_score:
                best_score, best_t = c, t

        chosen.append(best_t)
        remaining.remove(best_t)

    return chosen

selected_tickers = greedy_select(R_is, y_is, ridge_lambda, MAX_NAMES)
len(selected_tickers), selected_tickers[:10]



## 10. Rebuild the sector matrix for the selected set


In [ ]:

R_is_sel  = R_is[selected_tickers]
R_oos_sel = R_oos[selected_tickers]
sp500_sel = sp500[sp500["Ticker"].isin(selected_tickers)].reset_index(drop=True)

# Rebuild A for the selected universe
sectors_sel = sorted(sp500_sel["Sector"].unique())
row_sel = {s: i for i, s in enumerate(sectors_sel)}
S_sel = len(sectors_sel)
N_sel = len(selected_tickers)

A_sel = np.zeros((S_sel, N_sel), dtype=float)
for j, t in enumerate(selected_tickers):
    s = sp500_sel.loc[sp500_sel["Ticker"] == t, "Sector"].values[0]
    A_sel[row_sel[s], j] = 1.0

# Align target vector to sectors_sel
s_tgt_sel = np.array([sector_target.get(s, 0.0) for s in sectors_sel], dtype=float)
if s_tgt_sel.sum() > 0:
    s_tgt_sel = s_tgt_sel / s_tgt_sel.sum()

S_sel, N_sel, sectors_sel[:5]



## 11. Solve the sector-constrained Bayesian ridge QP

We minimize:
\[
\frac{1}{T}\lVert Rw - y \rVert^2 \; + \; \lambda \lVert w \rVert^2 \; + \; \gamma \lVert A w - s^* \rVert^2
\]
Subject to:
- \( \sum_i w_i = 1 \)
- Long-only (optional): \( w_i \ge 0 \)
- Per-name cap (optional): \( w_i \le c \)

We use **cvxpy** and the **OSQP** solver.


In [ ]:

def solve_tracker_qp(Rm: np.ndarray,
                     y: np.ndarray,
                     A: np.ndarray,
                     s_target: np.ndarray,
                     ridge_lambda: float,
                     gamma: float,
                     long_only: bool,
                     w_cap: float | None) -> np.ndarray:
    T, N = Rm.shape
    w = cp.Variable(N)

    # Tracking error
    resid = Rm @ w - y
    loss_track = (1.0 / T) * cp.sum_squares(resid)

    # Ridge regularization
    loss_ridge = ridge_lambda * cp.sum_squares(w)

    # Sector penalty
    loss_sector = 0
    if gamma > 0 and s_target.sum() > 0:
        loss_sector = gamma * cp.sum_squares(A @ w - s_target)

    objective = cp.Minimize(loss_track + loss_ridge + loss_sector)

    constraints = [cp.sum(w) == 1.0]
    if long_only:
        constraints += [w >= 0.0]
        if w_cap is not None:
            constraints += [w <= w_cap]

    prob = cp.Problem(objective, constraints)
    try:
        prob.solve(solver=cp.OSQP, eps_abs=1e-7, eps_rel=1e-7, verbose=False)
    except Exception:
        # Fallback solver
        prob.solve(solver=cp.SCS, verbose=False)

    if w.value is None:
        raise RuntimeError("Optimization failed. Try relaxing constraints or parameters.")
    return np.array(w.value).ravel()

weights = solve_tracker_qp(
    Rm=R_is_sel.values,
    y=y_is.values,
    A=A_sel,
    s_target=s_tgt_sel,
    ridge_lambda=ridge_lambda,
    gamma=SECTOR_PENALTY,
    long_only=LONG_ONLY,
    w_cap=PER_NAME_CAP if LONG_ONLY else None
)

weights_series = pd.Series(weights, index=R_is_sel.columns, name="Weight").sort_values(ascending=False)
weights_series.head(20)



## 12. Evaluate in-sample (IS) and out-of-sample (OOS) tracking

We compute portfolio returns, tracking error (TE), annualized TE volatility, beta, and R².


In [ ]:

def evaluate_tracking(R: pd.DataFrame, y: pd.Series, w: pd.Series) -> Tuple[pd.Series, Dict[str, float]]:
    rp = (R[w.index] @ w.values).rename("Portfolio")
    te = rp - y
    TE_vol = float(te.std() * np.sqrt(252))
    TE_mean = float(te.mean() * 252)
    R2 = float(np.corrcoef(rp, y)[0, 1]**2) if len(rp) > 2 else np.nan
    beta = float(np.cov(rp, y, ddof=1)[0, 1] / (rp.var(ddof=1) + 1e-12))
    return rp, {
        "TE_ann_vol": TE_vol,
        "TE_ann_mean": TE_mean,
        "R2": R2,
        "Beta": beta,
        "Names": int((w.values > 1e-6).sum())
    }

rp_is, met_is   = evaluate_tracking(R_is_sel, y_is, weights_series)
rp_oos, met_oos = evaluate_tracking(R_oos_sel, y_oos, weights_series)

print("In-sample metrics:", json.dumps(met_is, indent=2))
print("OOS metrics:", json.dumps(met_oos, indent=2))



## 13. Plot cumulative growth (IS and OOS)

We **do not** set explicit colors, and we keep each chart as a **single plot**, as a clean best practice.


In [ ]:

# In-sample cumulative growth
cum_is = pd.concat([rp_is.rename("Portfolio"), y_is.rename("Index")], axis=1).cumsum().apply(np.exp)
plt.figure(figsize=(9, 4))
plt.plot(cum_is.index, cum_is["Portfolio"], label="Portfolio")
plt.plot(cum_is.index, cum_is["Index"], label="Index")
plt.title("In-sample cumulative growth")
plt.ylabel("Growth of $1")
plt.legend()
plt.tight_layout()
plt.show()

# OOS cumulative growth
cum_oos = pd.concat([rp_oos.rename("Portfolio"), y_oos.rename("Index")], axis=1).cumsum().apply(np.exp)
plt.figure(figsize=(9, 4))
plt.plot(cum_oos.index, cum_oos["Portfolio"], label="Portfolio")
plt.plot(cum_oos.index, cum_oos["Index"], label="Index")
plt.title("Out-of-sample (~6 months) cumulative growth")
plt.ylabel("Growth of $1")
plt.legend()
plt.tight_layout()
plt.show()



## 14. Compare sector mix (Portfolio vs Target)

We compute the portfolio sector weights as `A @ w` and compare to the target vector aligned to `sectors_sel`.


In [ ]:

port_sector_weights = pd.Series(A_sel @ weights_series.loc[selected_tickers].values,
                                index=sectors_sel, name="Portfolio")
tgt_sector_weights  = pd.Series([sector_target.get(s, 0.0) for s in sectors_sel],
                                index=sectors_sel, name="Target")
if tgt_sector_weights.sum() > 0:
    tgt_sector_weights = tgt_sector_weights / tgt_sector_weights.sum()

sec_df = pd.concat([port_sector_weights, tgt_sector_weights], axis=1).fillna(0)

# Bar chart (two separate calls keep a single figure)
plt.figure(figsize=(10, 4))
# Portfolio
plt.bar(sec_df.index, sec_df["Portfolio"])
plt.title("Portfolio sector weights")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
# Target
plt.bar(sec_df.index, sec_df["Target"])
plt.title("Target sector weights")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

sec_df.sort_values("Target", ascending=False).head()



## 15. Simple 6-month forecast (AR(1) on index returns mapped by beta)

This **toy** forecast fits an AR(1) to **in-sample index returns**, then maps to the portfolio by multiplying with the **in-sample beta**.  
Replace with Black-Litterman, block bootstrap, or a macro model for more realism.


In [ ]:

from statsmodels.tsa.ar_model import AutoReg

# beta from in-sample
beta_is = met_is["Beta"]

# Fit AR(1) on in-sample index returns
ar = AutoReg(y_is.values, lags=1, old_names=False).fit()

steps = len(y_oos)  # forecast horizon ≈ OOS window length
y_fore = ar.predict(start=len(y_is), end=len(y_is) + steps - 1)
p_fore = beta_is * y_fore

df_fore = pd.DataFrame({"Index_fore": y_fore, "Portfolio_fore": p_fore})

cum_fore = df_fore.cumsum().apply(np.exp)
plt.figure(figsize=(9, 4))
plt.plot(cum_fore.index, cum_fore["Portfolio_fore"], label="Portfolio (forecast)")
plt.plot(cum_fore.index, cum_fore["Index_fore"], label="Index (forecast)")
plt.title("Forecast (~6 months) cumulative growth")
plt.ylabel("Growth of $1")
plt.legend()
plt.tight_layout()
plt.show()

df_fore.head()



## 16. Save weights and metrics

We write weights and metrics to CSV/JSON for downstream use.


In [ ]:

weights_df = (weights_series.to_frame()
              .merge(sp500_sel[["Ticker","Name","Sector","MarketCap"]],
                     left_index=True, right_on="Ticker")
              .sort_values("Weight", ascending=False))
weights_path = os.path.join(OUTPUT_DIR, "weights.csv")
weights_df.to_csv(weights_path, index=False)

metrics = {
    "in_sample": met_is,
    "oos": met_oos,
    "lambda": ridge_lambda,
    "gamma": SECTOR_PENALTY,
    "long_only": LONG_ONLY,
    "per_name_cap": PER_NAME_CAP,
    "lookback_years": LOOKBACK_YEARS,
    "oos_months": OOS_MONTHS,
    "universe_cap": UNIVERSE_CAP,
    "K": MAX_NAMES,
    "sectors": ALLOWED_SECTORS
}
metrics_path = os.path.join(OUTPUT_DIR, "metrics.json")
with open(metrics_path, "w") as f:
    json.dump(metrics, f, indent=2)

weights_path, metrics_path



## 17. Extensions and next steps

- **Exact cardinality**: use a mixed-integer solver with binary variables for on/off holdings (slower).
- **Shorting**: allow negative weights and add an L1 (or gross exposure) constraint to control leverage.
- **Different priors**: try elastic-net or hierarchical priors instead of ridge.
- **Turnover control**: add a term penalizing \(\lVert w_t - w_{t-1} \rVert\) and simulate rebalancing.
- **Better forecast**: plug in Black-Litterman, bootstrapped scenarios, VAR, or macro-driven models.
- **Data refinements**: use float-adjusted market caps; handle corporate actions; align to true index rebalances.



## 18. Environment information (versions)


In [ ]:

import sys, platform
import sklearn, cvxpy, statsmodels
print("Python:", sys.version)
print("Platform:", platform.platform())
print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)
print("scikit-learn:", sklearn.__version__)
print("cvxpy:", cvxpy.__version__)
print("statsmodels:", statsmodels.__version__)
